# Tiling Algorithm

Here is where I play with the tiling algorithm

In [1]:
%matplotlib inline

In [2]:
def get_redshift(cat):
    
    z = (u.Quantity(cat['cz']) / c.c).to(u.dimensionless_unscaled)
    MK = cat['Ktmag'] - cosmo.distmod(z)
    
    return z

We can find samples here: https://dcc.ligo.org/P1500071/public

In [4]:
import healpy as hp
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm

from matplotlib import pyplot as plt
from astropy.cosmology import WMAP9 as cosmo
from astropy.table import Column
import astropy.units as u
import astropy.constants as c
from matplotlib import colors
import math as m
import numpy as np

from astropy.coordinates import EarthLocation
from pytz import timezone
from astroplan import Observer
from astropy.time import Time
import datetime

from astroquery.skyview import SkyView
from astropy.coordinates import SkyCoord
from astroquery.irsa_dust import IrsaDust
from astropy.io import fits as pyfits
import glob
#!pip install aplpy
#import aplpy

from astropy.utils.data import download_file

url = ('https://dcc.ligo.org/P1500071/public/10202_bayestar.fits.gz')
filename = download_file(url, cache=True)

from astroquery.vizier import Vizier

Vizier.ROW_LIMIT = -1
cat, = Vizier.get_catalogs('J/ApJS/199/26/table3')
len(cat)

z = get_redshift(cat)

#prob = hp.read_map(filename)

#prob, distmu, distsigma, distnorm = hp.read_map(filename, field=[0, 1, 2, 3])

prob, distmu, distsigma, distnorm = hp.read_map(filename, field=range(4))

npix = len(prob)
print(npix)
nside = hp.npix2nside(npix)
pixarea = hp.nside2pixarea(nside)
pixarea

/home/bernie/miniconda3/lib/python3.5/site-packages/scipy/integrate/quadpack.py:356: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)
/home/bernie/miniconda3/lib/python3.5/site-packages/numpy/lib/function_base.py:2287: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
/home/bernie/miniconda3/lib/python3.5/site-packages/astropy/cosmology/core.py:1414: RuntimeWarning: divide by zero encountered in log10
  val = 5. * np.log10(abs(self.luminosity_distance(z).value)) + 25.0


NSIDE = 512
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT
Ordering converted to RING
Ordering converted to RING
Ordering converted to RING
Ordering converted to RING
3145728


3.994741635118857e-06

In [5]:
cat_now = cat
print(type(cat))

<class 'astropy.table.table.Table'>


In [6]:
r = cosmo.luminosity_distance(z).to('Mpc').value
theta = 0.5*np.pi - cat_now['_DEJ2000'].to('rad').value
phi = cat_now['_RAJ2000'].to('rad').value
ipix = hp.ang2pix(nside, theta, phi)

/home/bernie/miniconda3/lib/python3.5/site-packages/scipy/integrate/quadpack.py:356: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)
/home/bernie/miniconda3/lib/python3.5/site-packages/numpy/lib/function_base.py:2287: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [7]:
dp_dV = prob[ipix]*distnorm[ipix]*norm(distmu[ipix], distsigma[ipix]).pdf(r) / pixarea

/home/bernie/miniconda3/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:872: RuntimeWarning: invalid value encountered in greater_equal
  return (self.a <= x) & (x <= self.b)
/home/bernie/miniconda3/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:872: RuntimeWarning: invalid value encountered in less_equal
  return (self.a <= x) & (x <= self.b)


In [8]:
cands = cat_now[(dp_dV >= np.nanpercentile(dp_dV,95)) & (r <= 50.0)]


/home/bernie/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in greater_equal
  if __name__ == '__main__':
/home/bernie/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in less_equal
  if __name__ == '__main__':


In [9]:
print(len(cands))

602


In [ ]:
cands['ID'][0]

In [ ]:
def isnt_in(coords, covered_coords):
    isnt_in = True
    if len(covered_coords) > 0:
        for cov_cord in covered_coords:
            if abs(coords[0]-cov_cord[0]) <= 5.0 and abs(coords[1]-cov_cord[1]) <= 5.0:
                isnt_in = False

    return isnt_in

In [ ]:

def get_tile_cands(cands):
    %%time
    tile_cands = []
    max_score = {}
    max_score['score'] = []
    max_score['coords'] = []
    ra_corr = cands['_RAJ2000']*np.cos(np.radians(cands['_DEJ2000']))
    k = 0
    foo = True
    for cand in cands:
        #if foo == True:
            if k%100 == 0:
                print('indexing... ' + str(k))

            ra = cand['_RAJ2000']
            dec = cand['_DEJ2000']
            coords = SkyCoord(ra, dec, frame='fk5', unit='deg')

            # Top Left
            top_left = {}
            top_left['name'] = 'Top Left on ' + str(cand['SimbadName'])
            top_left['ra_min'] = np.float64((ra - 0.05)*np.cos(coords.dec.to('radian')))
            top_left['ra_max'] = np.float64((ra + 2.55)*np.cos(coords.dec.to('radian')))
            top_left['dec_max'] = np.float64(dec + 0.05)
            top_left['dec_min'] = np.float64(dec - 1.95)
            top_left['center_ra'] = 0.5*(top_left['ra_min'] + top_left['ra_max'])
            top_left['center_dec'] = 0.5*(top_left['dec_min'] + top_left['dec_max'])

            # Top Right
            top_right = {}
            top_right['name'] = 'Top Right on ' + str(cand['SimbadName'])
            top_right['ra_max'] = np.float64((ra + 0.05)*np.cos(coords.dec.to('radian')))
            top_right['ra_min'] = np.float64((ra - 2.55)*np.cos(coords.dec.to('radian')))
            top_right['dec_max'] = top_left['dec_max']
            top_right['dec_min'] = top_left['dec_min']
            top_right['center_ra'] = 0.5*(top_right['ra_min'] + top_right['ra_max'])
            top_right['center_dec'] = top_left['center_dec']

            # Bottom Left
            bottom_left = {}
            bottom_left['name'] = 'Bottom Left on ' + str(cand['SimbadName'])
            bottom_left['ra_min'] = top_left['ra_min']
            bottom_left['ra_max'] = top_left['ra_max']
            bottom_left['dec_min'] = np.float64(dec - 0.05)
            bottom_left['dec_max'] = np.float64(dec + 1.95)
            bottom_left['center_ra'] = top_left['center_ra']
            bottom_left['center_dec'] = 0.5*(bottom_left['dec_min'] + bottom_left['dec_max'])

            # Bottom Right
            bottom_right = {}
            bottom_right['name'] = 'Bottom Right on ' + str(cand['SimbadName'])
            bottom_right['ra_max'] = top_right['ra_max']
            bottom_right['ra_min'] = top_right['ra_min']
            bottom_right['dec_min'] = bottom_left['dec_min']
            bottom_right['dec_max'] = bottom_left['dec_max']
            bottom_right['center_ra'] = top_right['center_ra']
            bottom_right['center_dec'] = bottom_left['center_dec']

            # Centered
            centered = {}
            centered['name'] = 'Centered on ' + str(cand['SimbadName'])
            centered['ra_min'] = np.float64((ra - 1.3)*np.cos(coords.dec.to('radian')))
            centered['ra_max'] = np.float64((ra + 1.3)*np.cos(coords.dec.to('radian')))
            centered['dec_max'] = np.float64(dec + 1.0)
            centered['dec_min'] = np.float64(dec - 1.0)
            centered['center_ra'] = ra*np.cos(np.radians(dec))
            centered['center_dec'] = dec

            cords = [top_left,top_right,bottom_left,bottom_right,centered]

            for cord in cords:
                tile = {}
                tile['gal_indexes'] = []
                center_coords = SkyCoord(float(cord['center_ra']), float(cord['center_dec']), 'fk5', unit='deg')

                tile['properties'] = {'name': cord['name'],
                                      'coords': center_coords.to_string('hmsdms'),
                                      'coords_num': [cord['center_ra'], cord['center_dec']],
                                      'score': 0}

                keep = (ra_corr<=cord['ra_max']) & (ra_corr>=cord['ra_min']) & (cands['_DEJ2000']<=cord['dec_max']) & (cands['_DEJ2000']>=cord['dec_min'])

                cands2 = cands[keep]

                tile['galaxies'] = []

                for cand_k in cands2:


                    cand_coords = SkyCoord(float(cand_k['_RAJ2000']*np.cos(np.radians(cand_k['_DEJ2000']))), float(cand_k['_DEJ2000']), frame='fk5', unit='deg')

                    tile['galaxies'].append({'name': cand_k['SimbadName'],
                                             'coords': cand_coords.to_string('hmsdms'),
                                             'anything else': 'other stuff'})

                    tile['gal_indexes'].append(cand_k['index'])

                tile['properties']['score'] = len(tile['galaxies'])

                if tile['properties']['score'] < 10:
                    foo = False

                #if tile['properties']['score'] >= max(max_score['score']):
                tile_cands.append(tile)
                max_score['score'].append(tile['properties']['score'])
                #print('New max score: ', max(max_score['score']))
                max_score['coords'].append(tile['properties']['coords_num'])

            k = k+1
        
    return tile_cands, max_score

In [ ]:
def get_tiles(cands, tile_cands, max_score, tiles):
    
    appended = 0
    
    new_cands = cands
    max_scores = np.array(max_score['score'])
    len_max = len(max_scores[max_scores >= np.nanpercentile(max_scores, 98)])

    scores = np.array(max_score['score'])
    coords = np.array(max_score['coords'])
    indexes = np.argsort(-scores)
    scores = scores[indexes]
    coords = coords[indexes]
    tile_cands = np.array(tile_cands)
    tile_cands = tile_cands[indexes]
    
    #if len_max >= 50:
    #    sort = {'score': scores[:50], 'coords': coords[:50]}
    #else:
        
    sort = {'score': scores[:len_max], 'coords': coords[:len_max]}

    print(sort)
    range_covered = []
    gal_indexes = []
    for j in range(len(tile_cands)):
        if len(tiles) < 16:
            tile = tile_cands[j]
        #print(tile['properties'])
            if tile['properties']['score'] in sort['score'] and isnt_in(tile['properties']['coords_num'], range_covered)==True:
                appended = appended + 1
                tiles.append(tile)
                if len(tile['gal_indexes']) > 0:
                    for ind in tile['gal_indexes']:
                        if ind not in gal_indexes:
                            gal_indexes.append(ind)
            #print('Appended!')
                range_covered.append(tile['properties']['coords_num'])

            
    galaxy_indexes = np.array(gal_indexes)
    new_cands.remove_rows(galaxy_indexes[np.argsort(-galaxy_indexes)])

    return new_cands

In [ ]:
def tile_sky(cands):
    
    cands = cat_now[(dp_dV >= np.nanpercentile(dp_dV,95)) & (r <= 50.0)]
    cands = cands[cands['_DEJ2000'] <= 18.0]
    
    
    
    tiles = []
    loop_cands = cands
    one_to_n = np.arange(len(loop_cands))
    idx = Column(name='index', data=one_to_n)
    loop_cands.add_column(idx, index=0)
    while len(tiles) < 16:
        print(len(tiles))
        print(str(len(loop_cands))+' candidates left')
        
        tile_cands, max_score = get_tile_cands(loop_cands)
        
        loop_cands = get_tiles(loop_cands, tile_cands, max_score, tiles)
        
        
        
        loop_cands.remove_column('index')
        one_to_n = np.arange(len(loop_cands))
        idx = Column(name='index', data=one_to_n)
        loop_cands.add_column(idx, index=0)
        
    return tiles

In [ ]:
cands = cat_now[(dp_dV >= np.nanpercentile(dp_dV,95)) & (r <= 50.0)]

cands = cands[cands['_DEJ2000'] <= 18.0]
print(len(cands))
len_cands = len(cands)



#cands.show_in_notebook()

In [ ]:
%%time

tiles = tile_sky(cands)

In [ ]:
print(len(tiles))
for tile in tiles:
    print(tile['properties'])

In [ ]:
cands.show_in_notebook()

In [ ]:
cands_hist = cat_now[(dp_dV >= np.nanpercentile(dp_dV,95)) & (r <= 50.0)]

cands_hist = cands_hist[cands_hist['_DEJ2000'] <= 18.0]
print(len(cands_hist))
len_cands = len(cands_hist)

org_ra = []
org_dec =[]

for ra_ind in range(len(cands_hist['_RAJ2000'])):
    org_dec.append(cands_hist['_DEJ2000'][ra_ind])
    org_ra.append(cands_hist['_RAJ2000'][ra_ind]*m.cos(m.radians(cands_hist['_DEJ2000'][ra_ind])))

In [ ]:
tl = 0
tr = 0
br = 0
bl = 0
cent = 0

for tile in tiles:
    if 'Top Left' in tile['properties']['name']:
        tl = tl + 1
    elif 'Top Right' in tile['properties']['name']:
        tr = tr + 1
    elif 'Bottom Left' in tile['properties']['name']:
        bl = bl + 1
    elif 'Bottom Right' in tile['properties']['name']:
        br = br + 1
    elif 'Centered' in tile['properties']['name']:
        cent = cent + 1

In [ ]:
fig, ax = plt.subplots()
n_groups = 5
index = np.arange(n_groups)
bar_width = 0.35

opacity = 0.7
error_config = {'ecolor': '0.3'}

rects1 = plt.bar(0.25 + index, [tl, tr, bl, br, cent], bar_width,
                 alpha=opacity,
                 color='b',
                 label='')


plt.xlabel('Group')
plt.ylabel('Scores')
plt.title('Scores By Group \n With ' + str(len_cands-len(cands)) + ' Galaxies Covered!\n')
plt.xticks(0.25 + index + 0.5*bar_width, ('Top Left', 'Top Right', 'Bottom Left', 'Bottom Right', 'Centered'))
plt.legend()

plt.tight_layout()
plt.show()

print(len(cands))

In [ ]:
decs = []
ras = []
cands_hist = cat_now[(dp_dV >= np.nanpercentile(dp_dV,95)) & (r<=50.0)]
cands_hist = cands_hist[cands_hist['_DEJ2000'] <= 18.0]
for tile in tiles:
    #for gal in tile['galaxies']:
        coords = SkyCoord(tile['properties']['coords_num'][0], tile['properties']['coords_num'][1], unit='deg', frame='fk5')
        decs.append(coords.dec.deg)
        ras.append(coords.ra.deg)
plt.figure(figsize=(20,10))
from matplotlib import cm
afm_cmap = cm.summer
afm_cmap.set_under("w") # sets background to white
plt.hist2d(org_ra, org_dec, bins = [144, 90],range=np.array([(0,360), (-90,90)]), cmap = afm_cmap)
plt.colorbar()

plt.scatter(ras, decs, color='b', s=40, alpha=.6, marker = 's')


plt.xlabel('RA (deg)', fontsize = 15)
plt.ylabel('DEC (deg)', fontsize = 15)
plt.title('Galaxy Method: Tiled Galaxies \n With ' + str(len_cands-len(cands)) + ' Galaxies Covered and Taking '
          + str(30) + ' Seconds Per Loop! \n', fontsize = 20)
plt.xlim([0,360])
plt.ylim([-90,90])

In [ ]:
decs = []
ras = []

for tile in tiles:
    for gal in tile['galaxies']:
        coords = SkyCoord(gal['coords'], frame='fk5')
        decs.append(coords.dec.deg)
        ras.append(coords.ra.deg)
plt.figure(figsize=(20,10))
from matplotlib import cm
afm_cmap = cm.summer
afm_cmap.set_under("w") # sets background to white
plt.hist2d(org_ra, org_dec, bins = [144, 90],range=np.array([(0,360), (-90,90)]), cmap = afm_cmap)
plt.colorbar()

plt.scatter(ras, decs, color='b', s=1, alpha=.7)


plt.xlabel('RA (deg)', fontsize = 15)
plt.ylabel('DEC (deg)', fontsize = 15)
plt.title('Galaxy Method: Groups of Tiled Galaxies \n With ' + str(len_cands-len(cands)) + ' Galaxies Covered and Taking '
          + str(30) + ' Seconds Per Loop! \n', fontsize = 20)
plt.xlim([0,360])
plt.ylim([-90,90])
#plt.colorbar()

## Histogram Method

In [ ]:
cands_hist = cat_now[(dp_dV >= np.nanpercentile(dp_dV,95)) & (r<=50.0)]
cands_hist = cands_hist[cands_hist['_DEJ2000'] <= 18.0]
print(len(cands_hist))

In [ ]:
#histo, xes, ys = plt.hist2d(cands_hist['_RAJ2000'], cands_hist['_DEJ2000'], bins = [144, 180],range=np.array([(0,360), (-90,90)]))

plt.figure(figsize=(20,10))
plt.hist2d(cands_hist['_RAJ2000'], cands_hist['_DEJ2000'], bins = [144, 90],range=np.array([(0,360), (-90,90)]))

plt.colorbar()



In [ ]:
histo, xes, ys, img = plt.hist2d(cands_hist['_RAJ2000'], cands_hist['_DEJ2000'], bins = [139, 90],range=np.array([(0,360), (-90,90)]))

In [ ]:
print(histo)

In [ ]:
foo= True
def find_score(histo):
#if foo == True:
    tls = {'score': [], 'row': [], 'colmn': []}

    for k in range(len(histo)):
        row = k
        colmn = np.where(histo[k]==max(histo[k]))
        for col in colmn:
            for co in col:
                tls['score'].append(max(histo[k]))
                tls['row'].append(row)
                tls['colmn'].append(co)

    tls['score']=np.array(tls['score'])
    tls['row']=np.array(tls['row'])
    tls['colmn']=np.array(tls['colmn'])

    indexes_of_max = np.argsort(-tls['score'])
    sorted_tls = {'score': tls['score'][indexes_of_max],
                  'row': tls['row'][indexes_of_max],
                  'colmn': tls['colmn'][indexes_of_max],
                  'score_overall': 0}

    #print(sorted_tls['score'][:16])
    #print(sorted_tls['colmn'][:16])
    #print(sorted_tls['row'][:16])
    sorted_tls['score_overall'] = sum(sorted_tls['score'][:15])
    #print(sorted_tls['score_overall'])
    
    return sorted_tls

In [ ]:
org_ra = []
org_dec =[]

for ra_ind in range(len(cands_hist['_RAJ2000'])):
    org_dec.append(cands_hist['_DEJ2000'][ra_ind])
    org_ra.append(cands_hist['_RAJ2000'][ra_ind]*m.cos(m.radians(cands_hist['_DEJ2000'][ra_ind])))

In [ ]:


%%time
ra_fov = 2.6
dec_fov = 2.0

N_steps_ra = 50
N_steps_dec = 20
tiles_histo = {}
tiles_histo['RA'] = []
tiles_histo['DEC'] = []
n_fin = -1
mn_fin = -1

max_score = -1

for n in range(N_steps_dec):
    
    min_dec = -90 + n*dec_fov/N_steps_dec
    max_dec = 90 + n*dec_fov/N_steps_dec
    
    for mn in range(N_steps_ra):
        
        min_ra = 0 + mn*ra_fov/N_steps_ra
        max_ra = 360 + mn*ra_fov/N_steps_ra
        
        histo, xes, ys, img = plt.hist2d(org_ra,
                                         org_dec, bins = [139, 90], 
                                         range=np.array([(min_ra,max_ra),
                                                         (min_dec,max_dec)]))
        sorted_tls = find_score(histo)
        
        if sorted_tls['score_overall'] > max_score:
            tiles_histo = sorted_tls
            max_score = sorted_tls['score_overall']
            print('New max score! ', max_score)
            n_fin  = n
            mn_fin = mn

tiles_histo['RA'] = []
tiles_histo['DEC'] = []
for ra_ind in range(len(tiles_histo['row'])):
    tiles_histo['DEC'].append(-90+mn*dec_fov/N_steps_dec + 180*(tiles_histo['colmn'][ra_ind]+0.5)/90)
    tiles_histo['RA'].append((mn*ra_fov/N_steps_ra + 360*(tiles_histo['row'][ra_ind]+0.5)/139))
                             #*m.cos(m.radians(tiles_histo['DEC'][ra_ind])))

In [ ]:
decs = []
ras = []

for tile in tiles:
    #for gal in tile['galaxies']:
        coords = SkyCoord(tile['properties']['coords_num'][0], tile['properties']['coords_num'][1], unit='deg', frame='fk5')
        decs.append(coords.dec.deg)
        ras.append(coords.ra.deg)



plt.figure(figsize=(20,10))
from matplotlib import cm
afm_cmap = cm.summer
afm_cmap.set_under("w") # sets background to white
plt.hist2d(org_ra, org_dec, bins = [144, 90],range=np.array([(0,360), (-90,90)]), cmap = afm_cmap)
plt.colorbar()

plt.scatter(tiles_histo['RA'][:17], tiles_histo['DEC'][:17], marker='s', color='r', s=40, alpha=.6)
plt.scatter(ras, decs, marker='s', color='b', s=40, alpha=.6)

plt.xlabel('RA (deg)', fontsize = 15)
plt.ylabel('DEC (deg)', fontsize=15)
plt.title('Tiles Selected: 10202_bayestar Sample With Distance < 50 Mpc\n With ' + str(int(tiles_histo['score_overall'])) 
          + ' Galaxies Covered and Taking ' + str(1.5) + ' Minutes by Histogram Method (RED)\n'+ 'With '
          + str(len_cands-len(cands)) + ' Galaxies Covered and Taking '+ str(30) +
          ' Seconds Per Loop by Galaxy Method (BLUE)\n', fontsize=20)
plt.xlim([0,360])
plt.ylim([-90,90])

In [ ]:
print(tiles_histo['score_overall'])
for k in range(len(tiles_histo['score'][:16])):
    print('score, ',tiles_histo['score'][k])


In [ ]:
all_gals = cat_now[(r<=50.0)]
all_gals = cands_hist[cands_hist['_DEJ2000'] <= 18.0]
print(len(all_gals))